In [45]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as matplot
from sklearn.decomposition import PCA
import random
from IPython.display import display, HTML
from itertools import chain
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sb
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC, LinearSVC
import warnings
warnings.filterwarnings('ignore')

Q1(a)(i) 
Load MNIST dataset. After necessary data preparation, build a linear SVM classifier. (5 points)

In [2]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

2022-11-05 15:29:02.862040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 15:29:03.006257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-05 15:29:03.006282: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-05 15:29:03.048860: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-05 15:29:03.937639: W tensorflow/stream_executor/platform/de

In [3]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
train_images = train_images/255.0

In [5]:
test_images = test_images/255.0

In [6]:
train_images = np.reshape(train_images, (60000, 784))
test_images = np.reshape(test_images, (10000, 784))

In [7]:
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [46]:
pca = PCA(n_components=33, whiten=True)
train_images = pca.fit_transform(train_images)
test_images = pca.transform(test_images)

In [47]:
svm = LinearSVC(dual=False)
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
accuracy_score(test_labels, pred) 

0.8889

ii. Show the predictions for first twenty values of your test dataset. Display the confusion matrix. (4
points)

In [48]:
for i in range(20):
    print(pred[i])

7
2
1
0
4
1
4
9
6
9
0
6
9
0
1
5
9
7
3
4


In [49]:
confmat = confusion_matrix(test_labels, pred)
print(confmat)

[[ 955    0    2    2    1    8    8    2    2    0]
 [   0 1108    3    2    0    3    4    1   14    0]
 [  11   10  879   21   15    4   18   20   46    8]
 [   4    1   24  892    1   37    5   12   21   13]
 [   4    3    7    0  889    6   10    2    6   55]
 [  21    3    5   57   27  691   26   14   35   13]
 [  17    3    6    2   11   14  899    2    4    0]
 [   6   14   29    1   14    0    0  933    3   28]
 [  16   10   12   33   12   40   15   13  807   16]
 [  12   10    6    9   54   29    1   39   13  836]]


Write your own function to calculate class-wise F1 score. 

In [17]:
type(confmat)

numpy.ndarray

In [50]:
from operator import le


prec_list = confmat.sum(axis=0)
recall_list = confmat.sum(axis=1)
precisions = []
recalls = []
for i in range(10):
    precisions.append(float(confmat[i][i])/prec_list[i])
    recalls.append(float(confmat[i][i])/recall_list[i])

print(precisions)
print(recalls)


[0.9130019120458891, 0.9535283993115319, 0.9033915724563206, 0.8753680078508341, 0.8681640625, 0.8305288461538461, 0.9117647058823529, 0.8988439306358381, 0.8485804416403786, 0.8627450980392157]
[0.9744897959183674, 0.9762114537444934, 0.8517441860465116, 0.8831683168316832, 0.905295315682281, 0.7746636771300448, 0.9384133611691023, 0.9075875486381323, 0.8285420944558521, 0.8285431119920713]


In [51]:
def f1(pr):
    (p,r) = pr
    return (2*(p*r)/(p+r))
f1s = list(map(f1, zip(precisions, recalls)))

print(f1s)

[0.9427443237907207, 0.9647366129734436, 0.8768079800498753, 0.8792508624938394, 0.8863409770687937, 0.8016241299303944, 0.9248971193415639, 0.9031945788964182, 0.8384415584415584, 0.845298281092012]


Check the F1 scores using sklearn inbuilt function and compare the value with the F1 scores returned
by your function written from scratch. Also, report the accuracy.

In [52]:
from sklearn.metrics import f1_score
f1s_scikit = f1_score(test_labels, pred, average=None)


In [53]:
print(f1s)
print(f1s_scikit)


[0.9427443237907207, 0.9647366129734436, 0.8768079800498753, 0.8792508624938394, 0.8863409770687937, 0.8016241299303944, 0.9248971193415639, 0.9031945788964182, 0.8384415584415584, 0.845298281092012]
[0.94274432 0.96473661 0.87680798 0.87925086 0.88634098 0.80162413
 0.92489712 0.90319458 0.83844156 0.84529828]


In [54]:
accuracy_score(test_labels, pred)

0.8889

(b) Non linear SVM:
Build non-linear models with the RBF kernel as well as polynomial kernel. Report the accuracy. (5 points)

In [55]:
svm = SVC(kernel='rbf')
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
print(accuracy_score(test_labels, pred) )

0.9816


In [56]:
svm = SVC(kernel='poly')
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
print(accuracy_score(test_labels, pred) )

0.9802


In [58]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV


# specify range of hyperparameters
# Set the parameters by cross-validation
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [5,10]}]


# specify model
model_svc = SVC(kernel="rbf")

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = model_svc, 
                        param_grid = hyper_params, 
                        scoring= 'accuracy', 
                        cv = 5, 
                        return_train_score=True,n_jobs=-1)      

# fit the model
model_cv.fit(pd.DataFrame(train_images).sample(frac=0.5,random_state=2), pd.DataFrame(train_labels).sample(frac=0.5,random_state=2))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [61]:
print(model_cv.best_score_)
print(model_cv.best_params_)

0.9757333333333333
{'C': 10, 'gamma': 0.01}


1(d)-build the final model with chosen hyperparameters. Report confusion matrix, accuracy

In [64]:
from sklearn import metrics

model_best = SVC(C=10, gamma=0.01, kernel="rbf")
model_best.fit(pd.DataFrame(train_images).sample(frac=0.5,random_state=2), pd.DataFrame(train_labels).sample(frac=0.5,random_state=2))
y_pred = model_best.predict(test_images)

print("accuracy", metrics.accuracy_score(test_labels, y_pred), "\n")
print(metrics.confusion_matrix(test_labels, y_pred), "\n")

accuracy 0.9762 

[[ 972    0    2    0    0    1    1    1    2    1]
 [   1 1129    1    1    0    0    1    1    1    0]
 [   4    3 1005    1    2    0    1   10    5    1]
 [   0    0    3  985    0    8    0    8    4    2]
 [   2    2    4    0  953    0    3    1    1   16]
 [   3    0    1    9    0  873    1    1    3    1]
 [   6    2    1    0    5    4  936    1    3    0]
 [   0    8   13    2    0    0    0  999    0    6]
 [   2    0    2    7    4    4    1    4  947    3]
 [   4    4    0    7   12    7    0    6    6  963]] 



Q1 - (e)

In [67]:
#df_x_train = pd.DataFrame(x_train.reshape(-1, 784))
df_x_train_sv = pd.DataFrame(train_images).iloc[model_best.support_]
print("sv x-train df")
display(df_x_train_sv)
df_y_train_sv = pd.DataFrame(train_labels).iloc[model_best.support_]
print("sv y-train df")
display(df_y_train_sv)

sv x-train df


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
5,0.319802,0.448732,1.285846,0.120752,0.801338,0.808000,-0.094415,0.313504,-0.152826,0.385639,...,-1.820543,-0.984886,-0.306983,-1.306892,-1.325258,-0.651355,-0.104699,-1.303714,-1.430368,0.250358
56,2.294076,0.021281,-0.639979,-0.502469,-1.723804,1.434764,0.973948,-2.107978,1.542318,0.207205,...,-0.495132,1.378227,0.095164,-1.753846,1.554485,-0.101968,-0.208043,-1.108610,-0.388647,2.818461
148,-0.700765,0.632395,0.474044,0.165393,0.359867,0.236551,0.312470,-0.482769,-1.660833,-0.193065,...,-0.174054,1.003151,-1.112420,0.805269,0.391175,0.490304,0.045258,-0.775300,1.561168,-0.003438
159,0.341488,-0.106663,0.668699,0.982034,0.984839,1.755431,0.710409,-0.396602,-0.133186,1.126085,...,1.139030,-1.909130,0.694019,0.577981,0.160698,-0.813433,-0.271831,-0.628725,-0.756666,1.696593
362,-0.513831,1.613086,0.002887,-0.641181,0.672257,-0.726260,0.093434,-0.443538,-0.704793,0.649866,...,-1.716197,1.423122,-1.335225,0.862446,-0.440255,0.625936,1.226752,-0.701345,0.122208,-2.147576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29503,0.957106,0.788084,-1.033672,0.699591,-0.257587,0.117391,-2.704520,1.701084,1.895972,-1.999785,...,1.034368,-0.796359,-0.108388,0.448244,2.158050,-1.286618,-1.533533,0.991896,2.085807,-2.424059
29730,-1.025864,0.277623,0.182794,1.272015,-1.192859,-0.534704,-0.894572,-0.727299,0.282813,-0.757105,...,-0.889170,0.360880,-0.856711,-1.079449,-0.109660,0.040884,0.645604,-1.495391,0.594210,1.038756
29781,-0.919668,-0.139007,1.318588,-0.214717,-1.066033,-0.019375,-0.509173,0.140657,1.056732,-1.360548,...,1.510828,2.065887,-0.519124,-0.029540,0.602503,0.451471,2.378685,0.251800,0.212291,0.456999
29854,0.139923,1.363941,-0.362641,1.879431,-0.079606,1.092127,-1.891040,1.092463,0.312300,-0.604564,...,-0.148259,0.524891,-0.451714,-1.169831,0.398186,-0.557387,0.917255,-0.031470,1.724624,-1.717456


sv y-train df


,0
5,2
56,0
148,7
159,2
362,9
...,...
29503,4
29730,7
29781,5
29854,4


In [71]:
model_sv = SVC(C=10, gamma=0.01, kernel="rbf")

model_sv.fit(df_x_train_sv,df_y_train_sv)
y_pred_train = model_sv.predict(train_images)
y_pred_test = model_sv.predict(test_images)

print("accuracy over train set", metrics.accuracy_score(train_labels, y_pred_train), "\n")
print("accuracy over test set", metrics.accuracy_score(test_labels, y_pred_test), "\n")

accuracy over train set 0.9585333333333333 

accuracy over test set 0.9555 

